In [1]:
import time
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# ✅ Step 1: Set up Headless Chrome
options = Options()
options.add_argument("--headless")           # Run in background
options.add_argument("--disable-gpu")        # Avoid GPU rendering issues
options.add_argument("--window-size=1920x1080")

# Initialize the WebDriver
driver = webdriver.Chrome()  # This creates the 'driver' object
driver.get("https://www.amazon.com")
time.sleep(3)

In [2]:
# 🧠 Step 2: Search for "Airpods Best Sellers"
search_box = driver.find_element(By.XPATH, '//input[@id="twotabsearchtextbox"]')
search_box.send_keys("Apple Watch Series")
search_box.send_keys(Keys.RETURN)           

In [3]:
page = 1
all_products = []

with open('Amazon_AppleWatch_data.csv', mode='w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(['Product_name', 'Price', 'Rating', 'Reviews', 'Product URL'])

    while True:
        base_url = f"https://www.amazon.com/s?k=Apple+Watch+Series&crid=WE78EG888JPM&sprefix=apple+watch+series%2Caps%2C744&page={page}&ref=nb_sb_noss_2"
        driver.get(base_url)
        time.sleep(2)

        # ✅ 
        products = driver.find_elements(By.XPATH, '//div[@data-component-type="s-search-result"]')
        print(f"\n Page {page} — Products found: {len(products)}")  

        if len(products) == 0:
            print("No more products found. Scraping complete.")
            break

        for product in products:
            try:
                product_name = product.find_element(By.XPATH, './/h2[@class="a-size-medium a-spacing-none a-color-base a-text-normal"]/span').text
                try:
                    price_whole = product.find_element(By.XPATH, './/span[@class="a-price-whole"]').text
                    price_fraction = product.find_element(By.XPATH, './/span[@class="a-price-fraction"]').text
                    price = f"${price_whole}.{price_fraction}"
                except:
                    price = "N/A"

                link = product.find_element(By.XPATH, './/a[@class="a-link-normal s-line-clamp-2 s-link-style a-text-normal"]').get_attribute('href')
                full_link = link
                    
                try:
                    rating = product.find_element(By.XPATH, './/span[contains(@class,"a-icon-alt")]').get_attribute('innerHTML').strip()
                except:
                    rating = "N/A"

                try:
                    reviews = product.find_element(By.XPATH, './/span[@class="a-size-base s-underline-text"]').text
                except:
                    reviews = "N/A"
                    
                all_products.append([product_name, price, rating, reviews, full_link])
                writer.writerow([product_name, price, rating, reviews, full_link])
                print(f"Product: {product_name} | Price: {price} | Link: {full_link} | Rating: {rating} | Reviews: {reviews}")

            except Exception as e:
                print(f"Error extracting product: {e}")

        page += 1  # ✅ Page increase

# ✅ Convert to DataFrame
df = pd.DataFrame(all_products, columns=["Product Name", "Price", "Rating", "Reviews", "Product URL"])
df.to_csv("Amazon_AppleWatch_data.csv", index=False, encoding="utf-8-sig")
print("\nData saved to Amazon_AppleWatch_data.csv and loaded into pandas DataFrame.")


 Page 1 — Products found: 16
Product: Watch Series 9 GPS 41mm Midnight Aluminum Case with Midnight Sport Band - S/M (2023) | Price: N/A | Link: https://www.amazon.com/Apple-Watch-GPS-Aluminum-Midnight/dp/B0CHX3PBRX/ref=sr_1_1?crid=WE78EG888JPM&dib=eyJ2IjoiMSJ9.kioLUs8vWON_Yby8q1mBTdNcTzFVpm-SzbTA5At2JNNBk8Iu-Q6T9n43NaP9pqgmUSJCttVix3PXqEDsw2O9YEC9kJ_3H5sfP-8Nb0_NSvEgp3ssSFGJBWZ5_Uq-d-l76mv4yFi3QDI2Dkuzr31tzeGPHJNl5pU1AhFO22jIdtERGo8ibIKHeY-8Hm2mdKtCpDEVbE3qVa_tFFDd6Ed5lKCIj-ofUaveH42JeTj7QRM.0Jd_rkYCdigUTMUIX8HD6bj7thlwk229P9J9DDx8uH4&dib_tag=se&keywords=Apple+Watch+Series&qid=1748785082&sprefix=apple+watch+series%2Caps%2C744&sr=8-1 | Rating: 4.7 out of 5 stars | Reviews: 499
Product: Watch SE (2nd Gen) [GPS 40mm] Smartwatch with Starlight Aluminum Case with Starlight Sport Band S/M. Fitness & Sleep Tracker, Crash Detection, Heart Rate Monitor | Price: N/A | Link: https://www.amazon.com/Apple-Smartwatch-Starlight-Aluminum-Detection/dp/B0CHX7R6WJ/ref=sr_1_2?crid=WE78EG888JPM&dib=eyJ2Ij